# Fine-tuning BERT on CSI data with MLM

In [1]:
import torch
from torch.optim import AdamW
import pandas as pd
from glob import glob
import numpy as np
from transformers import BertTokenizer, BertForMaskedLM
from tqdm.auto import tqdm

In [2]:
# load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# get path to all sub files
sub_files = glob('cleaned_subs/*.tsv')

# return all subs lines as list
def data_loader(files):
    'loads texts into a single list'
    texts = []
    for f in sub_files:
        df = pd.read_csv(f, sep='\t', usecols=['text'])
        texts += list(df["text"].values)
    return texts

data = data_loader(sub_files)
len(data)

216856

In [4]:
print(data[0])
print(data[-1])

My name is Royce Harmon.
You should watch it.


In [5]:
# tokenize the data with the BERT tokenizer
inputs = tokenizer(data, 
    max_length=512,
    truncation=True,
    padding=True,
    return_tensors='pt')

print(inputs.keys())
print(inputs['input_ids'].shape)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([216856, 91])


In [6]:
inputs.input_ids[0]

tensor([  101,  2026,  2171,  2003, 14789, 25546,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0])

In [7]:
# deep copy the inputs to labels (because we will mask the inputs)
inputs['labels'] = inputs['input_ids'].detach().clone()
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [8]:
# mask with 15% probability

# create random tensor(0-1) of size inputs
random_tensor = torch.rand(inputs['input_ids'].shape)
# create a mask with 15% selected while also ingnoring
# [cls] == 101, [sep] == 102, and [pad] == 0
masked_tensor = (random_tensor < 0.15)*(inputs['input_ids'] != 101)*(inputs['input_ids'] != 102)*(inputs['input_ids'] != 0)

# check sizes align
masked_tensor.shape, inputs['input_ids'].shape

(torch.Size([216856, 91]), torch.Size([216856, 91]))

In [9]:
# getting indices where masked is True
non_zero_indices = []
for mask in masked_tensor:
    non_zero_indices.append(torch.flatten(mask.nonzero()).tolist())

print(non_zero_indices[:20])

[[], [1, 7, 12, 13], [7, 8], [3], [4, 6, 17, 18], [], [4], [], [6, 7, 8], [1, 3, 7], [], [1], [2, 6, 8, 9], [], [], [], [2], [17, 20, 21], [3], [4]]


In [10]:
# replace masked values with 103
for i in range(len(inputs['input_ids'])):
    inputs['input_ids'][i, non_zero_indices[i]] = 103

In [11]:
## pytorch dataset class

class CSIDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, index):
        input_ids = self.encodings['input_ids'][index]
        attention_mask = self.encodings['attention_mask'][index]
        token_type_ids = self.encodings['token_type_ids'][index]
        labels = self.encodings['labels'][index]
        return {
            'input_ids':input_ids, 
            'attention_mask':attention_mask,
            'token_type_ids':token_type_ids,
            'labels': labels
        }


In [12]:
# create torch dataset
dataset = CSIDataset(inputs)

In [13]:
# create torch data loader
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16,
    shuffle=True
    )

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [15]:
model.to(device);

In [16]:
epochs = 32
optimizer = AdamW(model.parameters(), lr=1e-5)

model.train();


In [17]:

# training loop
for epoch in range(epochs):
    loop = tqdm(data_loader)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        # print(input_ids.shape)
        labels = batch['labels']
        attention_mask =  batch['attention_mask']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch: {epoch}')
        loop.set_postfix(loss=loss.item())


Epoch: 31: 100%|██████████| 1/1 [00:06<00:00,  6.51s/it, loss=5.46]


In [18]:
model.save_pretrained('model/bert-base-uncased-csi-trained');


In [19]:
# model = BertForMaskedLM.from_pretrained("bert-base-uncased");
model.from_pretrained('model/bert-base-uncased-csi-trained');


def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = f"[CLS] {text} [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("\t[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

        
predict_masked_sent("You should [MASK] it.", top_k=5)


	[MASK]: 'do'  | weights: 0.05219556391239166
	[MASK]: 'try'  | weights: 0.049260374158620834
	[MASK]: 'find'  | weights: 0.03673092648386955
	[MASK]: 'take'  | weights: 0.03608653321862221
	[MASK]: 'enjoy'  | weights: 0.0333879292011261


In [20]:
import random
for sent in random.sample(data, 5):
    print(sent)
    words = sent.split(' ')
    mask_int = np.random.randint(0,len(words),1)
    words[mask_int[0]] = '[MASK]'
    masked_sent = ' '.join(words)
    print(masked_sent)
    predict_masked_sent(masked_sent, top_k=5)


Anne-Marie stopped by the lab to discuss a case.
Anne-Marie [MASK] by the lab to discuss a case.
	[MASK]: 'stops'  | weights: 0.43644979596138
	[MASK]: 'drops'  | weights: 0.11503851413726807
	[MASK]: 'stopped'  | weights: 0.06933876127004623
	[MASK]: 'arrives'  | weights: 0.06736624240875244
	[MASK]: 'comes'  | weights: 0.02786053903400898
Dragged him into the bathroom.
[MASK] him into the bathroom.
	[MASK]: 'pushed'  | weights: 0.06020032614469528
	[MASK]: 'shoved'  | weights: 0.04883961379528046
	[MASK]: 'follow'  | weights: 0.04824935272336006
	[MASK]: 'toss'  | weights: 0.04525557905435562
	[MASK]: 'drag'  | weights: 0.03403385356068611
That's crazy.
[MASK] crazy.
	[MASK]: 'fucking'  | weights: 0.16949908435344696
	[MASK]: 'totally'  | weights: 0.06971689313650131
	[MASK]: 'freaking'  | weights: 0.050160106271505356
	[MASK]: 'completely'  | weights: 0.043558623641729355
	[MASK]: 'like'  | weights: 0.035368070006370544
It's from the Latin word for magpie, a bird with a large and in

## Getting the sentence embedding

In [21]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.from_pretrained('model/bert-base-uncased-csi-trained');

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def sentence_embedder(sentences):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


# sentences = data[:5]
# sentence_embeddings = sentence_embedder(sentences)
# print(data[:5])
# print("Sentence embeddings:")
# print(sentence_embeddings)




In [22]:
sentence_embeddings[0].shape

NameError: name 'sentence_embeddings' is not defined

In [ ]:


def ls_data__embedder(path):
    '''loads labeled sub files, embeds the sentences, and saves
    embedded episodes and labels into binary files
    '''
    # get path to all labels_sub files
    sub_files = glob(path + '/*.tsv')
    
    # loads texts 
    for f in sub_files:
        print(f)
        # read with pandas
        df = pd.read_csv(f, sep='\t', usecols=['text', 'label'])
        # get sentences as list
        data = list(df["text"].values)
        # convert labels to 0 and 1
        labels = [0 if v == 'N' else 1 for v in df["label"]]

        # embed sentences and save episode as numpy array
        np.savetxt(f[:-4]+'_emb.bin', sentence_embedder(data).numpy())
        # save labels as numpy array
        np.savetxt(f[:-4]+'_lab.bin', np.array(labels))

ls_data_embedder('labeled_subs')

    